In [ ]:
import os
import sys
import urllib.request
import urllib.parse
import pandas as pd
import tkinter as tk
from tkinter import filedialog, scrolledtext, messagebox
import json
import time
import threading
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from concurrent.futures import ThreadPoolExecutor

# API 인증 정보 (기존과 동일)
api_credentials = [
    {"client_id": "2OVaobyxNhphELuzT7lK", "client_secret": "Z89woersg3"},
    {"client_id": "6iFGXDV7meV6ZRCjzIUX", "client_secret": "Zpk81osYLB"},
    {"client_id": "Ds_TslJUTyf8Xgj6SjSq", "client_secret": "JkHgqMo72j"},
    {"client_id": "DuJZQMutx_P2XoJNsdQj", "client_secret": "CXLsaJn9sy"},
    {"client_id": "ik_lIfJLL7YAxRAe0uii", "client_secret": "Jm88iAEl2i"},
    {"client_id": "VSAWLVnOtERmTwrcaGLg", "client_secret": "HwdjxHxx6k"},
    {"client_id": "Zw1ifDEOgOHX5jNLjsj8", "client_secret": "qW9ASPbnzW"},
    {"client_id": "MoxBWJpnFUKz6bStk13B", "client_secret": "OeSzRjEZDP"},
    {"client_id": "BjmVyS4ZNuQGcJf0YcRf", "client_secret": "12QwGPYCId"},
    {"client_id": "w9HgLHVHnjDVQA6tuztM", "client_secret": "OYEeYr7PYk"},
    {"client_id": "a1mfMF8CENNs4Qg0awep", "client_secret": "XO5O1RQo4x"},
    {"client_id": "k8kVw3Lo9zGV3c1oGMWv", "client_secret": "U_YDPn_JhR"},
    {"client_id": "kAmiaQ4vJYYg7cEAO_Ca", "client_secret": "BsRZ21mMD9"},
    {"client_id": "t1TpVGeAzhmA23dXLLxT", "client_secret": "YcObvJLPqM"},
    {"client_id": "jwizLVNRd6rfeSKeB6QQ", "client_secret": "MJ5REntCic"},
    {"client_id": "zdToVAoR4cjCqBHs70lO", "client_secret": "KseNWmC9dp"},
    {"client_id": "YF2tBWKrCbJkaM3fGBcu", "client_secret": "lmMIV7vXx9"},
    {"client_id": "91tyZsTJP3A4HF7HPTX9", "client_secret": "J5AxYKAHJV"},
    {"client_id": "xqjyps4nhKLBDpum1o8q", "client_secret": "Ym16K0GEV0"},
    {"client_id": "ghzkzgQYgPTkrrffGDP1", "client_secret": "ewVcXxtDDs"}
]

# 파일 선택, 엑셀 데이터 읽기, 링크 열고 닫기, 상태 업데이트 함수 (기존과 동일)
def select_excel_file():
    root = tk.Tk()
    root.withdraw()
    file_path = filedialog.askopenfilename(
        title="엑셀 파일 선택",
        filetypes=(("Excel files", "*.xlsx;*.xls"), ("All files", "*.*"))
    )
    return file_path

def get_product_data_from_excel(file_path):
    try:
        df = pd.read_excel(file_path)
        product_data = df[['상품명', '판매자관리코드']]
        return product_data
    except Exception as e:
        update_status(f"엑셀 파일 읽기 오류: {e}")
        sys.exit(1)

def open_link_and_close_after_delay(link):
    chrome_driver_path = r"C:\Users\jh\클릭커대체\chromedriver.exe"
    service = Service(chrome_driver_path)
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(service=service, options=options)
    driver.get(link)
    time.sleep(0)
    driver.quit()

def update_status(message, is_blue=False):
    elapsed_time = time.time() - start_time
    minutes, seconds = divmod(int(elapsed_time), 60)
    time_str = f"[{minutes:02}:{seconds:02}]"

    if is_blue:
        status_text.tag_configure("blue", foreground="blue")
        status_text.insert(tk.END, f"{time_str} {message}\n", "blue")
    else:
        status_text.insert(tk.END, f"{time_str} {message}\n")

    status_text.yview(tk.END)
    root.update_idletasks()

# 네이버 API 검색 (수정된 함수)
def search_naver_blog(query, product_name, seller_code, credential_index):
    print(f"Thread {threading.current_thread().name}: Searching for '{product_name}' with seller_code '{seller_code}'") # [PRINT 1] 함수 시작 시 product_name 확인
    encText = urllib.parse.quote(query)
    url = f"https://openapi.naver.com/v1/search/shop?query={encText}"

    credentials = api_credentials[credential_index % len(api_credentials)]
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id", credentials["client_id"])
    request.add_header("X-Naver-Client-Secret", credentials["client_secret"])

    try:
        response = urllib.request.urlopen(request)
        rescode = response.getcode()
        if rescode == 200:
            response_body = response.read()
            json_data = json.loads(response_body.decode('utf-8'))

            if json_data['items']:
                for item in json_data['items']:
                    title = item['title']
                    link = item['link']
                    if product_name in title and "11st" in link:
                        open_link_and_close_after_delay(link)
                        return True, seller_code
            return False, seller_code
        else:
            update_status(f"Error Code: {rescode}")
            return False, seller_code
    except Exception as e:
        update_status(f"네이버 검색 오류: {e}")
        return False, seller_code

# 검색 실행 (수정된 함수)
def run_search(file_path):
    product_data = get_product_data_from_excel(file_path)
    total_products = len(product_data)
    opened_links_count = 0
    not_opened_count = 0
    not_opened_products = []
    not_opened_products_lock = threading.Lock()

    update_status(f"총 {total_products}개 상품 검색 시작.")

    with ThreadPoolExecutor(max_workers=5) as executor:
        futures = []
        for index, row in product_data.iterrows():
            product_name = row['상품명']
            seller_code = row['판매자관리코드']
            update_status(f"{index+1}/{total_products}: '{product_name}' 검색 중...", is_blue=True)
            print(f"MainThread: Submitting task for '{product_name}' with seller_code '{seller_code}'") # [PRINT 2] 스레드 제출 전 product_name 확인
            futures.append(executor.submit(search_naver_blog, product_name, product_name, seller_code, index))

        for future in futures:
            product_name_for_result = product_data.iloc[futures.index(future)]['상품명'] # [수정] future에 해당하는 product_name 가져오기
            result, seller_code = future.result()
            print(f"MainThread: Received result for '{product_name_for_result}' with seller_code '{seller_code}', result: {result}") # [PRINT 3] 스레드 결과 수신 후 product_name 확인
            if result:
                opened_links_count += 1
            else:
                not_opened_count += 1
                with not_opened_products_lock:
                  not_opened_products.append({'상품명': product_name_for_result, '판매자관리코드': seller_code}) # [수정] 올바른 product_name 사용


    update_status(f"\n총 상품수: {total_products}")
    update_status(f"링크 연 상품 수: {opened_links_count}")
    update_status(f"열지 않은 상품 수: {not_opened_count}")

    if not_opened_products:
        not_opened_df = pd.DataFrame(not_opened_products)
        if not not_opened_df.empty:
            output_file_path = 'not_opened_11번가.xlsx'
            not_opened_df.to_excel(output_file_path, index=False)
            update_status(f"열지 않은 상품 목록: '{output_file_path}'")
        else:
            update_status("열지 않은 상품 없음.")

# GUI (기존과 동일)
def start_gui():
    global root, status_text, start_time
    root = tk.Tk()
    root.title("상품 검색")

    def on_closing():
        if messagebox.askokcancel("종료", "종료하시겠습니까?"):
            root.destroy()
            sys.exit(0)

    root.protocol("WM_DELETE_WINDOW", on_closing)

    status_text = scrolledtext.ScrolledText(root, width=150, height=60)
    status_text.pack(padx=10, pady=10)

    file_path = select_excel_file()
    if file_path:
        start_time = time.time()
        search_thread = threading.Thread(target=run_search, args=(file_path,))
        search_thread.start()
    else:
        update_status("파일 선택 안 함.")

    root.mainloop()

if __name__ == "__main__":
    start_gui()

MainThread: Submitting task for '끊김없이 꼬임없이 중형견 외출 리드줄 애견 앞섬 보더콜리 노령 노견 포메' with seller_code 'Jdmpet_1000012040'
Thread ThreadPoolExecutor-0_0: Searching for '끊김없이 꼬임없이 중형견 외출 리드줄 애견 앞섬 보더콜리 노령 노견 포메' with seller_code 'Jdmpet_1000012040'
MainThread: Submitting task for '130x80cm 체온유지 극세사 무릎 캠핑 차박 담요 아기 차량 블랑켓 회사 용' with seller_code 'JHSdmtopia_1309928'
Thread ThreadPoolExecutor-0_1: Searching for '130x80cm 체온유지 극세사 무릎 캠핑 차박 담요 아기 차량 블랑켓 회사 용' with seller_code 'JHSdmtopia_1309928'
MainThread: Submitting task for '130x70cm 이중보온 차량소품 무릎담요 무드캠핑 극세사담요 이쁜 단체 입는' with seller_code 'JHSdmtopia_1309479'
Thread ThreadPoolExecutor-0_2: Searching for '130x70cm 이중보온 차량소품 무릎담요 무드캠핑 극세사담요 이쁜 단체 입는' with seller_code 'JHSdmtopia_1309479'
MainThread: Submitting task for '감성충만 힐링충전 카페인테리어 업소장식 불멍소품 LED 무드등 커플 자취' with seller_code 'JHSdmtopia_1320329'
Thread ThreadPoolExecutor-0_3: Searching for '감성충만 힐링충전 카페인테리어 업소장식 불멍소품 LED 무드등 커플 자취' with seller_code 'JHSdmtopia_1320329'
MainThread: Submitting task for 